# Import Libraries

In [1]:
import os
import sys
import shutil
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam


import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F
#import torchvision.transforms.functional as F
#from torchvision.datasets import OxfordIIITPet

from PIL import Image
import io
from io import BytesIO 

import cv2

from tqdm import tqdm
from tqdm.notebook import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from albumentations import (
    Compose, OneOf, Normalize, CenterCrop, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, RandomRotate90, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, HueSaturationValue, CoarseDropout,GridDropout
    )

from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import Image as Im
from PIL import ImageOps

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Configurations

In [2]:
 class CFG:
        BASE_PATH = '../input/oxfordiiitpet-dataset/'
        IMAGE_PATH = '../input/oxfordiiitpet-dataset/images/images/'
        ANNOTATION_PATH = '../input/oxfordiiitpet-dataset/annotations/annotations/'
        
        DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        IMAGE_SIZE = 128
        BATCH_SIZE = 32
        
        def to_numpy(tensor):
            return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Create DataFrame

In [3]:
all_df = pd.read_table(CFG.ANNOTATION_PATH+'list.txt',sep=' ',skiprows=5 )
all_df = all_df.iloc[:,:4]
all_df = all_df.set_axis(['image_name','id','species','breed'],axis=1)
all_df

In [4]:
DEBUG = False
if DEBUG:
    all_df = all_df.sample(frac = 0.3).reset_index(drop = True)

In [5]:
all_df

# Split train and validation 8:2

In [6]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(all_df, test_size = 0.2)
print(train_df.shape, valid_df.shape)

In [7]:
train_df = train_df.reset_index(drop=True)
train_df

In [8]:
valid_df = valid_df.reset_index(drop=True)
valid_df

# Visualize images

In [9]:
#nomal images
image = Im.open(CFG.IMAGE_PATH+all_df["image_name"][0]+".jpg")
#image = Im.open('../input/oxfordiiitpet-dataset/images/images/Abyssinian_10.jpg')
display(image)
i = np.array(image)
i.shape

In [10]:
#mask image
#mask_image = PIL.ImageOps.autocontrast(load_img('../input/oxfordiiitpet-dataset/annotations/annotations/trimaps/Abyssinian_10.png'))
mask_image = PIL.ImageOps.autocontrast(load_img(CFG.ANNOTATION_PATH+'trimaps/'+all_df["image_name"][0]+'.png'))
display(mask_image)
rgb2gry = transforms.Grayscale()
mask_images = mask_image.convert('L')
#mask_image = mask_image.transpose(1,0,2)
mask = np.array(mask_image)
#print(mask)
#mask = mask.transpose(1,0,2)

obj_ids = np.unique(mask)
print(obj_ids)

# convert [0,127,255] to [0,1,2]
mask[mask == 0] = 1
mask[mask == 127] = 0
mask[mask == 255] = 1
#print(mask)
print(np.unique(mask))
#obj_ids[:]
#masks = mask == obj_ids[:,None,None]

In [11]:
print(type(image))
print(type(mask_image))

# image and mask

In [12]:
def get_concat_h(im1, im2):
    dst = Im.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

get_concat_h(image, mask_image)

# Augmentation function

In [13]:
def get_transform(data):
    if data == 'train':
        return A.Compose([
                A.Resize(CFG.IMAGE_SIZE,CFG.IMAGE_SIZE),
                A.HorizontalFlip(p=0.5),
                #A.GridDropout(ratio=0.2, unit_size_min=None, unit_size_max=None, holes_number_x=5, holes_number_y=5, shift_x=0, shift_y=0, random_offset=False, fill_value=0, mask_fill_value=None, always_apply=False, p=0.5),
                A.Normalize(),
                ToTensorV2()
            ])
    elif data == 'valid':
        return A.Compose([
                A.Resize(CFG.IMAGE_SIZE,CFG.IMAGE_SIZE),
                A.Normalize(),
                ToTensorV2()
            ])

In [14]:
len(all_df)

In [15]:
#confirm path for dataset class
image_path = CFG.IMAGE_PATH
mask_path = CFG.ANNOTATION_PATH
print(os.path.join(image_path, all_df["image_name"][0]+'.jpg'))
print(os.path.join(mask_path, 'trimaps/'+all_df["image_name"][0]+'.png'))

# Dataset

In [16]:
class TrainDataset(Dataset):
    def __init__(self, image_paths, mask_paths, df, transforms=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.df = df
        self.image_names = df["image_name"]
        self.transforms = transforms
        
    def __len__(self):
        #print(len(self.df))
        return len(self.df)
    
    def __getitem__(self,idx):
        # get path
        image_name = self.image_names[idx]
        image_path = os.path.join(self.image_paths, image_name+'.jpg')
        mask_path = os.path.join(self.mask_paths, 'trimaps/'+image_name+'.png')
        #print(image_path,mask_path)
        #open image
        image = Im.open(image_path)
        mask_images = PIL.ImageOps.autocontrast(load_img(mask_path))
        rgb2gry = transforms.Grayscale()
        mask_images = rgb2gry(mask_images)
        #PIL to numpy
        image = np.array(image)
        image = image[:,:,:3]#remove some of alpha channels
        mask_images = np.array(mask_images)
        #convert [0,127,255] -> [0,1,2]
        mask_images[mask_images == 127] = 1
        mask_images[mask_images == 255] = 2

        obj_ids = np.unique(mask_images)
        
        if self.transforms:
            augmented = self.transforms(image=image,mask=mask_images)
            image,mask = augmented['image'],augmented['mask']

        mask  = mask.unsqueeze(0)

        return image,mask

In [17]:
train_dataset = TrainDataset(
    CFG.IMAGE_PATH,
    CFG.ANNOTATION_PATH,
    train_df,
    transforms = get_transform(data='train'),
)
valid_dataset = TrainDataset(
    CFG.IMAGE_PATH,
    CFG.ANNOTATION_PATH,
    valid_df,
    transforms = get_transform(data='valid'),
)

In [18]:
# CPUのコア数を確認
import os
os.cpu_count()  # コア数

In [19]:
train_loader = DataLoader(train_dataset, CFG.BATCH_SIZE, shuffle = False, num_workers=os.cpu_count(), pin_memory=True)
train_dataset[0]

In [20]:
valid_loader = DataLoader(valid_dataset, CFG.BATCH_SIZE, shuffle = False, num_workers=os.cpu_count(), pin_memory=True)
valid_dataset[0]

# Define U-Net model

## part of U-Net models

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)
        
    def forward(self, x1,x2):
        x1 = self.up(x1)
        
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        
        x1 = F.pad(x1, [diffX // 2, diffX - diffX //2,
                        diffY // 2, diffY - diffY //2])
        
        x = torch.cat([x2,x1],dim=1)
        return self.conv(x)
    
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

In [21]:
#pytorch official
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1,bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
        #self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        #self.conv = DoubleConv(in_channels, out_channels)
        
    def forward(self, x1,x2):
        x1 = self.up(x1)
        
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        
        x1 = F.pad(x1, [diffX // 2, diffX - diffX //2,
                        diffY // 2, diffY - diffY //2])
        
        x = torch.cat([x2,x1],dim=1)
        return self.conv(x)
    
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

## main part of U-Net

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64,128)
        self.down2 = Down(128,256)
        self.down3 = Down(256,512)
        self.down4 = Down(512,1024)
        self.up1 = Up(1024,512)
        self.up2 = Up(512,256)
        self.up3 = Up(256,128)
        self.up4 = Up(128,64)
        self.outc = OutConv(64,n_classes)
    
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5,x4)
        x = self.up2(x,x3)
        x = self.up3(x,x2)
        x = self.up4(x,x1)
        logits = self.outc(x)
        return logits

In [22]:
#pytorch official
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes,bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64,128)
        self.down2 = Down(128,256)
        self.down3 = Down(256,512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512,1024 // factor)
        self.up1 = Up(1024,512 // factor, bilinear)
        self.up2 = Up(512,256 // factor, bilinear)
        self.up3 = Up(256,128 // factor, bilinear)
        self.up4 = Up(128,64, bilinear)
        self.outc = OutConv(64,n_classes)
    
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5,x4)
        x = self.up2(x,x3)
        x = self.up3(x,x2)
        x = self.up4(x,x1)
        logits = self.outc(x)
        return logits

# Custom metrics

In [23]:
def Dice_Coeff(x,y):
    
    #x:predmask, y:truemask
    x = CFG.to_numpy(x)
    y = CFG.to_numpy(y)
    x[x==2] = 0
    y[y==2] = 0
    
    x = torch.tensor(x)
    y = torch.tensor(y)
    
    intersection = torch.sum(x&y)
    
    dice = (2*intersection)/(CFG.IMAGE_SIZE*CFG.IMAGE_SIZE*CFG.BATCH_SIZE + CFG.IMAGE_SIZE*CFG.IMAGE_SIZE*CFG.BATCH_SIZE)

    return dice

# Visualize image (input and mask)

In [24]:
def show_pred_mask(original,true_tensor,out_tensor,cnt):
        unnormalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
        std=[1/0.229, 1/0.224, 1/0.255]
        )
        original = unnormalize(original)

        y = CFG.to_numpy(true_tensor)
        z = CFG.to_numpy(out_tensor)

        original = original.permute(2,1,0)
        original = CFG.to_numpy(original)
        original = np.rot90(original)
        original = np.flipud(original)

        y[y==0] = 255
        y[y==1] = 127
        y[y==2] = 0

        z[z==0] = 255
        z[z==1] = 127
        z[z==2] = 0

        n_data = 3
        row=1
        col=3
        fig, ax = plt.subplots(nrows=row, ncols=col,figsize=(15,18))

        #input image
        ax[0].imshow(original)

        #truth mask
        ax[1].imshow(y,cmap='Greys')

        #pred mask
        ax[2].imshow(z,cmap='Greys')

        plt.show()

# Training

In [25]:
plot_train_loss = []
plot_train_dice = []

plot_valid_loss = []
plot_valid_dice = []

L_t = []
D_t = []
L_v = []
D_v = []

def training_model(model, datasets, dataloaders, criterion, optimizer, num_epochs, device):
    best_model_weights = copy.deepcopy(model.state_dict())
    best_dice = 0.0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-'*10)
        
        for phase in ['train','valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_dice = 0.0
            
            stream = tqdm(dataloaders[phase])
            for cnt, (inputs, masks) in enumerate(stream, start=1):

                original = inputs
                inputs = inputs.to(device=CFG.DEVICE, dtype=torch.float32)
                masks = masks.to(device=CFG.DEVICE, dtype=torch.long)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    model = model.to(CFG.DEVICE)
                    outputs = model(inputs)
                    
                    pred_masks = F.softmax(outputs, dim=1).float()
                    masks =  masks.squeeze(1)
                    loss = criterion(pred_masks, masks)#CrossEntropyLoss
                    _,ch = torch.max(outputs, -3)
                    dice = Dice_Coeff(ch, masks)

                    running_loss += loss.item()
                    running_dice += dice
                    
                    if phase =='train':
                        L_t.append(loss.item())
                        D_t.append(dice)
                    else:
                        L_v.append(loss.item())
                        D_v.append(dice)

                    if cnt-1 == 0:
                        show_pred_mask(original[0],masks[0],ch[0],cnt)
                        show_pred_mask(original[1],masks[1],ch[1],cnt)
                    else:
                        continue
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
            epoch_loss = running_loss/len(dataloaders[phase])
            epoch_dice = running_dice/len(dataloaders[phase])
            
            if phase == 'train':
                plot_train_loss.append(epoch_loss)
                plot_train_dice.append(epoch_dice)
            else:
                plot_valid_loss.append(epoch_loss)
                plot_valid_dice.append(epoch_dice)
            
            print(f'{phase} Loss: {epoch_loss} Dice: {epoch_dice}')
            
            #if phase == 'valid' and epoch_dice > best_dice:
                #best_dice = epoch_dice
                #best_model_weights = copy().deepcopy(model.state_dict())
                
        print()
        
    #print(f'Best val Dice:{best_dice}')
          
    #model.load_state_dict(best_model_weights)
    
    return model

In [26]:
datasets = {'train': train_dataset,
            'valid': valid_dataset}

dataloaders = {'train': train_loader,
               'valid': valid_loader}
model = UNet(n_channels=3, n_classes=3)
#optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
num_epochs = 30

In [27]:
trained_model = training_model(model,datasets, dataloaders, criterion, optimizer, num_epochs, CFG.DEVICE)

In [28]:
len(L_t),len(D_t)

In [29]:
len(L_v),len(D_v)

In [34]:
fig,ax = plt.subplots()
bp = ax.boxplot(list(np.array_split(L_t, num_epochs)))
plt.title('Loss')
plt.ylabel('loss')
plt.ylim([0.7, 1.2])
plt.grid()

In [36]:
fig,ax = plt.subplots()
bp = ax.boxplot(list(np.array_split(D_t, num_epochs)))
plt.title('Dice')
plt.ylabel('dice')
plt.ylim([0, 0.7])
plt.grid()

In [32]:
tensorx = torch.tensor([[[1,0,1,1],[1,1,1,1],[1,0,0,1],[1,1,0,0]],[[1,0,1,0],[0,0,1,1],[0,0,0,1],[0,1,0,0]]])
tensory = torch.tensor([[[1,0,0,1],[1,0,0,1],[1,1,0,1],[1,0,0,0]],[[0,1,0,1],[0,1,0,1],[0,1,1,1],[1,0,0,0]]])
#tensorx = CFG.to_numpy(tensorx)
#tensory = CFG.to_numpy(tensory)
print(tensorx.shape,tensory.shape)
print(torch.sum(tensorx & tensory))
#print((torch.matmul(tensorx,tensory)).shape)
#print(sum(torch.matmul(tensorx,tensory)))
#print((sum(torch.matmul(tensorx,tensory))).shape)
#print(torch.sum(torch.matmul(tensorx,tensory)))

print(torch.sum(tensorx),torch.sum(tensory))

print((2*torch.sum(tensorx & tensory))/(4*4*2+4*4*2))